In [28]:
# Множители для отладки
PP = 1  #множитель перед членом с давлением
GG = 1    #гравитационная постоянная (приравнять нулю, чтобы посмотреть только с давлением)
RRRho = 1#e22  #множитель увеличивающий начальную плотность в центральной граничной точке
TT = 1  # Множитель перед температурой в центре

import time

import IPython
from IPython.display import display, HTML

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.patches import *
from md import *

%matplotlib inline

import math
import numpy as np

import matplotlib.gridspec as gridspec
#-------------------------------------------------------------------------------------#
mpl.use("nbAgg")
plt.style.use(['seaborn-v0_8-notebook'])
# plt.style.use(['seaborn-notebook'])
plt.rcParams.update({"figure.facecolor": (0,0,0,0),
                     "figure.dpi": 90,
                     "animation.html": "jshtml",
                     "animation.embed_limit": 100})
plt.rcParams['font.size'] = 20

# Задание различных физических констант
z = 3119 #3119 # Красное смещение
k = 1.38e-23 # Постоянная Больцмана Дж/К
m_p = 1.67e-27 # Масса протона кг
pk_to_meter = 3e16 # Переводной коэффициент пк->м
G = GG * 6.67e-11 # гравитационная постоянная Н*м^2/кг^2
M_pbh = 1e9 # масса черной дыры в солнечных массах
M_sun = 2e30 # масса солнца в кг
gamma = 5/3 # коэффициент адиабаты

t_end_years = 1e9 # Конечное время симуляции в годах
t_end_seconds = 3.154e7 * t_end_years
draw_step = 10000 # Шаг для отрисовки

# Средняя плотность Вселенной
# Rho_mean = 8.5e-27 * (5.1e-5 * (1+z)**4 + 0.308 * (1+z)**3 + 0.691 + 0.000949 * (1+z)**2) # в кг/м**3
Rho_mean = 8.5e-27 * 0.308 * (1+z)**3
# Блок констант далее требуется для условия Куранта-Фридрихсона-Леви (КФЛ)
mu = 1 # Средний молекулярный вес, это значение для водорода, если будем что-то добавлять то будет что-то другое
a1 = 1/119  # первый коэффициент в формуле для температуры
a2 = 1/115  # второй коэффициент в формуле для температуры
T = 2.7 * (1+z) * (1 + a1**(-1)*(1+z)**(-1)/(1+a2**(3/2)*(1+z)**(3/2))) ** (-1)
a = np.sqrt(k*T / (mu*m_p))   #скорость звука в м/c


# Радиус остановки DM
r_s = 6.7 * 10**22 * (M_pbh/10**9)**(1/3) * ((1+z)/18)**(-4/3) * 3.24 * 10**(-19)
r_virial = r_s/4

# Создаем одномерную пространственную сетку
# Диапазон сетки
r_min = r_virial#1 # внутренняя граница сетки в парсеках
r_max = 10*r_s#100 # внешняя граница сетки в парсеках

# Логарифмическая сетка
# r_phys - физические координаты узлов сетки
# r_log - десятичные логарифмы координат сетки
r_log_min = np.log(r_min)
r_log_max = np.log(r_max)
length_r = 50
r_log = np.linspace(r_log_min, r_log_max, length_r)
r_phys = np.exp(1) ** r_log
log_step = r_log[1]-r_log[0]

# Задание первичного (пробного) условия КФЛ
# Далее условие будет пересчитываться
# Вид условия: С = ((a + |u_x|) * dt)/dx < 0.5
# Здесь a-эффективная скорость звука, u_x-скорость газа, dt и dx шаги по времени и пространству
# Константа в условии КФЛ
C = 0.1


# Для логарифмической сетки
# Берем самое начало сетки, так как dt здесь будет минимально
dt = C * (r_phys[1]-r_phys[0]) * pk_to_meter / a
# Чтобы отрисовать далее решение нужно начать выполнение этого блока, а затем остановить когда нужно
# Подстройка массивов отрисовки происходит автоматически

# Создаем массивы для хранения различных переменных
U = np.zeros((2, length_r)) # скорость газа U_0 = 0
Rho = np.zeros((2, length_r)) # плотность газа Rho
P = np.zeros((2, length_r)) # давление газа P
T = np.zeros(length_r) # температура газа T
Eps = np.zeros(length_r) # внутренняя энергия газа U
E = np.zeros((2, length_r)) # Полная энергия
n_H = np.zeros(length_r) # количественная плотность водорода
Phi = np.zeros(length_r)  # гравитационный потенциал

r_log_temp_2 = np.append(r_log[0] - log_step, r_log)
r_log_temp_2 = np.append(r_log_temp_2, r_log[-1] + log_step)
r_temp_2 = np.exp(1) ** r_log_temp_2
Phi_temp = (-G*mdmt(r_temp_2[0]/r_s)/(r_temp_2[0] * pk_to_meter) * (Rho_mean * 4*np.pi/3 * (r_s*pk_to_meter)**3)
              -G*M_pbh*M_sun/(r_temp_2[0] * pk_to_meter))
Phi_temp_2 = (-G*mdmt(r_temp_2[-1]/r_s)/(r_temp_2[-1] * pk_to_meter) * (Rho_mean * 4*np.pi/3 * (r_s*pk_to_meter)**3)
              -G*M_pbh*M_sun/(r_temp_2[-1] * pk_to_meter))

#начальные условия
for j in range(length_r):
    # Гравитационный потенциал
    Phi[j] = (-G*mdmt(r_phys[j]/r_s)/(r_phys[j] * pk_to_meter) * (Rho_mean * 4*np.pi/3 * (r_s*pk_to_meter)**3)
              -G*M_pbh*M_sun/(r_phys[j] * pk_to_meter)) # в м**2/с**2

    # Температура
    T[j] = 2.7 * (1+z) * (1 + a1**(-1)*(1+z)**(-1)/(1+a2**(3/2)*(1+z)**(3/2)))**(-1)  # в кельвинах


    # Плотность барионов
    # omega_b = 0.049
    # omega_m = 0.308
    # Rho[0, j] = omega_b * rrb(r_phys[j]/r_s)/omega_m    #пробная плотность в Rho_mean
    Rho[0, j] = rrb(r_phys[j]/r_s)
    # Так как переопределили Rho и оставили метры то нужно переопределить единицы массы
    M_unit = Rho_mean   # Коэффициент перевода единиц массы в кг


    # Концентрация водорода
    n_H[j] = (Rho[0, j])/(m_p/M_unit)


    # массовая плотность внутренней энергия
    Eps[j] = (n_H[j] * 3/2 * k/M_unit * T[j])/(Rho[0, j])  # В Дж/M_unit


    # Скорости
    U[0, j] = 0   # В м/c


    # Полная энергия
    E[0, j] = (Rho[0, j]) * (U[0, j]**2/2 + Eps[j]) # В M_unit/м*с**2


    # Уравнение состояния
    P[0, j] = (gamma - 1) * (Rho[0, j]) * Eps[j]   # Давление в M_unit/м*с**2




# Расчет решения
# Вычисление значений массивов S, V, S_dual и V_dual (площади и обьемы шаровых слоев)
S_dual = np.zeros(length_r)
V_dual = np.zeros(length_r)

r_log_temp = np.append(r_log, r_log[-1] + log_step)
r_temp = np.exp(1) ** r_log_temp
r_log_temp_2 = r_log_temp - 0.5*log_step
r_temp_3 = np.exp(1) ** r_log_temp_2

S = np.zeros(length_r)
V_dual = np.zeros(length_r + 1)
V = np.zeros(length_r + 1)
S_dual = np.zeros(length_r)

# Сферический случай
S = 4 * np.pi * (r_temp_3[1:]+r_temp_3[:-1])**2 / 4 * pk_to_meter**2
V = 4/3 * np.pi * (r_temp_3[1:]**3 - r_temp_3[:-1]**3) * pk_to_meter**3
S_dual = 4 * np.pi * (r_temp[1:] + r_temp[:-1])**2 / 4 * pk_to_meter**2
V_dual = 4/3 * np.pi * (r_temp_3[1:]**3 - r_temp_3[:-1]**3) * pk_to_meter**3

# Включение гравитационной энергии в полную
# m = Rho[0,:] * V
# E[0,:] += m * Phi

Phi = np.append(Phi_temp, Phi)
Phi = np.append(Phi, Phi_temp_2)
P[0, -1] = P[0, -2]
U[0, -1] = U[0, -2]

# Искуственное увеличение плотности в центре
Rho[0,0] = Rho[0,0]*RRRho
T[0] = T[0]*TT

In [54]:
# Функции для расчета излучения
def eddington_light(n_H, x, E):  # Не учитывается рекомбинация так как светит во все стороны (соответственно зависимости от температуры здесь нет)
    # Количество свободных электронов
    n_e = x * n_H
    n_H_0 = (1-x) * n_H
    # Количество нейтрального водорода

    L_edd = 10**40 * M_pbh/(1e9)

    E_0 = 13.6   # в эВ
    sigma_photoionization = (E/E_0)**(-3) * 6e-22 * (E>E_0)  # м**2 (с обрезкой меньше 13.6 эВ)
    k_ion = n_H_0 * sigma_photoionization
    sigma_scat = 6.652e-29  # м**2

    H = np.zeros(r_phys.size)
    H[0] = L_edd/(4*np.pi*(r_phys[0]*pk_to_meter)**2)
    # print('k_ion',k_ion)
    # print('n_e*sigma_scat', n_e*sigma_scat)

    for i in range(1,r_phys.size-1):
        H[i] = (r_phys[i-1]/r_phys[i])**2 * H[i-1] * (1 - (r_phys[i]-r_phys[i-1]) * pk_to_meter * (k_ion[i] + n_e[i]*sigma_scat))
        # print((r_phys[i]-r_phys[i-1]) * pk_to_meter * (k_ion[i] + n_e[i]*sigma_scat))
    H = H*np.where(H>0,1,0)
    H = H*np.where(H<=H[0],1,0)
    for i in range(1,r_phys.size-1):
        if H[i]>H[i-1]:
            H[i] = 0
    return H

def ioniz_frac(dt, x, n_H, E, T, H):
    # Темп ионизации
    E_0 = 13.6
    n_H_0 = (1-x)*n_H
    n_e = x*n_H
    sigma_photoionizatioh = (E/E_0)**(-3)*6*10**(-22) # м**2, здесь E_0=13.6, а E-энергия фотонов, также здесь нужна обрезка энергии, чтобы она была только больше E_0
    ioniz_rate = n_H_0*sigma_photoionizatioh*H

    # Темп рекомбинации
    recombination_coef = 2.6*10**(-13)*T/10**4 * 1e-12 # м**3/cек значение из статьи Кара
    recombination_rate = n_e**2 * recombination_coef  # 1/сек

    # Вычисляем приросты
    n_H_0 += dt*(recombination_rate - ioniz_rate)
    n_e += dt*(ioniz_rate - recombination_rate)

    for i in range(n_e.size):
        if n_e[i]<0:
            n_e[i] = 0
            n_H_0[i] = n_H[i]
        elif n_e[i]>n_H[i]:
            n_e[i] = n_H[i]
            n_H_0[i] = 0

        if n_H_0[i]<0:
            n_H_0[i] = 0
            n_e[i] = n_H[i]
        elif n_H_0[i]>n_H[i]:
            n_H[i] = n_H_0
            n_e[i] = 0

    x = n_e/n_H

    return x

# def euler_coupling():


In [55]:
x = np.exp(-5*r_phys/r_s)
print(eddington_light(n_H,x,2000))
# print(x)

[ 2.80401428e+04  1.10142879e+04  4.33239380e+03  1.70591678e+03
  6.72192339e+02  2.64945004e+02  1.04406074e+02  4.11085020e+01
  1.61596544e+01  6.33575567e+00  2.47454043e+00  9.61288231e-01
  3.70724586e-01  1.41603780e-01  5.34176362e-02  1.98321193e-02
  7.21582895e-03  2.55970010e-03  8.79684891e-04  2.90610682e-04
  9.13957957e-05  2.70303501e-05  7.40022665e-06  1.83680500e-06
  4.01691416e-07  7.42554917e-08  1.08680502e-08  1.11797979e-09
  6.04517427e-11 -0.00000000e+00  0.00000000e+00 -0.00000000e+00
  0.00000000e+00 -0.00000000e+00  0.00000000e+00 -0.00000000e+00
  0.00000000e+00 -0.00000000e+00  0.00000000e+00 -0.00000000e+00
  0.00000000e+00 -0.00000000e+00  0.00000000e+00 -0.00000000e+00
  0.00000000e+00 -0.00000000e+00  0.00000000e+00 -0.00000000e+00
  0.00000000e+00  0.00000000e+00]


In [36]:
r_phys

array([  5.61543886,   6.05450727,   6.52790622,   7.03832   ,
         7.58864279,   8.18199505,   8.82174123,   9.51150884,
        10.25520905,  11.05705882,  11.92160482,  12.85374925,
        13.8587776 ,  14.94238862,  16.11072665,  17.37041646,
        18.7286008 ,  20.19298091,  21.77186018,  23.47419124,
        25.30962674,  27.28857403,  29.42225425,  31.72276587,
        34.20315337,  36.87748114,  39.7609133 ,  42.8697996 ,
        46.22176819,  49.83582557,  53.73246432,  57.93377933,
        62.46359311,  67.34759081,  72.61346589,  78.29107715,
        84.41261806,  91.01279924,  98.12904536, 105.80170726,
       114.07429084, 122.99370366, 132.610521  , 142.97927256,
       154.15875173, 166.21234888, 179.20841088, 193.22062859,
       208.32845472, 224.61755437])

In [11]:
r_phys*pk_to_meter

array([1.68463166e+17, 1.90374258e+17, 2.15135207e+17, 2.43116678e+17,
       2.74737548e+17, 3.10471174e+17, 3.50852478e+17, 3.96485961e+17,
       4.48054743e+17, 5.06330797e+17, 5.72186503e+17, 6.46607703e+17,
       7.30708466e+17, 8.25747760e+17, 9.33148301e+17, 1.05451785e+18,
       1.19167328e+18, 1.34666777e+18, 1.52182155e+18, 1.71975664e+18,
       1.94343608e+18, 2.19620828e+18, 2.48185721e+18, 2.80465893e+18,
       3.16944574e+18, 3.58167838e+18, 4.04752790e+18, 4.57396794e+18,
       5.16887919e+18, 5.84116732e+18, 6.60089632e+18, 7.45943917e+18,
       8.42964804e+18, 9.52604672e+18, 1.07650480e+19, 1.21651996e+19,
       1.37474612e+19, 1.55355190e+19, 1.75561399e+19, 1.98395720e+19,
       2.24199977e+19, 2.53360454e+19, 2.86313676e+19, 3.23552946e+19,
       3.65635726e+19, 4.13191985e+19, 4.66933629e+19, 5.27665158e+19,
       5.96295709e+19, 6.73852663e+19])

In [12]:
du_dr = np.zeros(length_r)
U_half = np.zeros(length_r)

# Инициализация счетчика шагов по времени
i = 0

# Инициализация массивов для отрисовки
U_draw = U[0, :]
Rho_draw = Rho[0, :]
E_draw = E[0, :]
draw_time = 0

# Инициализация переменной для хранения времени симуляции
sim_time = 0

# Задание множителя который будет использован при пересчете шагов по времени
dt_minimizer = 0.0009

# Индикатор нужный для пересчета шага по времени
indicator_2 = False

# Переменная для хранения плотности которая уходит к ЧД за пределы сетки
Rho_dump_left = np.zeros(2)

# Иницализация переменной для хранения разницы членов гравитации и давления
LL = np.zeros(length_r)

while (sim_time < t_end_seconds):
    if (i == 0):
        i = 1

    # Лагранжев шаг

    # Вычисление скорости на полшага вперед по времени
    dP_dx = (P[0,1:] - P[0,:-1]) / log_step
    dPhi_dx = (Phi[1:] - Phi[:-1]) / log_step
    LL[:-1] = (- PP * (0.5*dt * dP_dx/(r_phys[:-1] * pk_to_meter)) / (Rho[0,:-1])
                                - (0.5*dt * dPhi_dx[1:-1]/(r_phys[:-1] * pk_to_meter)))
    U_half[:-1] = U[0,:-1] + LL[:-1]
    U_half[-1] = U_half[-2]

    # Вычисление производной скорости по времени
    du_dx = (U_half[1:] - U_half[:-1]) / log_step
    du_dr[:-1] = du_dx / (r_phys[:-1] * pk_to_meter)
    du_dr[-1] = du_dr[-2]

    # Промежуточная полная энергия
    E_inter = E[0,:] - PP * P[0,:]*du_dr*0.5*dt - Rho[0,:] * U_half[:] * dPhi_dx[:-1]/(r_phys * pk_to_meter) * 0.5*dt

    # Шаг адвекции
    # В зависимости от направления скорости переносим параметры (плотность, энергию) из одной ячейки в другую

    # Вычисление скорости
    U[1,:] = U[0,:] - np.sign(U_half) * (dt * U_half ** 2 * S_dual) / V_dual
    # U[1,:] = np.sign(U_half) * np.abs(np.abs(U[0,:]) - dt * U_half ** 2 * S_dual) / (V_dual)
    U[1,:-1] =  U[1,:-1] + np.sign(U_half[1:])*( dt * U_half[1:]**2 * S_dual[1:]) / (V_dual[1:]) * np.where(U_half[1:]<0, 1, 0)
    U[1,1:] = U[1,1:] + np.sign(U_half[:-1])*( dt * U_half[:-1]**2 * S_dual[:-1]) / (V_dual[:-1]) * np.where(U_half[:-1]>0, 1, 0)
    U[1,-1] = U[1,-2]

    # Вычисление плотности
    Rho[1,1:] = Rho[0,1:] - np.abs((dt * Rho[0,1:] * U_half[1:] * S[1:]) / V[1:])
    # Следующую строчку нужно использовать чтобы вещество вылетало к центру. Ту что далее тогда нужно закомментировать
    Rho[1,0] = Rho[0,0] - np.abs((dt * Rho[0,0] * U_half[0] * S[0]) / V[0])
    # Следующую строчку нужно использовать если нужно накапливать плотность крайней точке. Ту что до этого тогда нужно закомментировать
    # Rho[1,0] = Rho[0,0] - np.abs((dt * Rho[0,0] * U_half[0] * S[0]) / V[0])*(U_half[0]>0)
    Rho_dump_left[1] = Rho_dump_left[0] + np.abs((dt * Rho[0,0] * U_half[0] * S[0]) / V[0]) * (U_half[0]<0)
    Rho[1,:-1] = Rho[1,:-1] + np.abs((dt * Rho[0,1:] * U_half[1:] * S[1:]) / V[1:]) * np.where(U_half[1:]<0, 1, 0)
    Rho[1,1:] = Rho[1,1:] + np.abs((dt * Rho[0,:-1] * U_half[:-1] * S[:-1]) / V[:-1]) * np.where(U_half[:-1]>0, 1, 0)

    # Вычисление полной энергии
    E[1,:] = E[0,:] - dt * E_inter * U_half * S / V
    # E[1,:] = np.abs(E[0,:]) - np.abs((dt * E_inter * U_half * S) / V)
    E[1,:-1] = E[1,:-1] + ((dt * E_inter[1:] * U_half[1:] * S[1:]) / V[1:]) * np.where(U_half[1:]<0, 1, 0)
    E[1,1:] = E[1,1:] + ((dt * E_inter[:-1] * U_half[:-1] * S[:-1]) / V[:-1]) * np.where(U_half[:-1]>0, 1, 0)

    # Расчет внутренней энергии
    # m = Rho[1,:] * V
    Eps = (E[1,:]/Rho[1,:] - U[1,:]**2/2) #- m*Phi[:-2]
    n_H = Rho[1,:]/(m_p/M_unit)
    T = Rho[1,:]*Eps/(n_H*3/2*k/M_unit)
    # if i==1: print(T[0])


    # Условный оператор для остановки расчета при получении некорректных значений
    if np.any(np.isnan(Rho[1,:])):
        print(U_half)
        break

    # Обновление шага по времени, чтобы он отвечал условию КФЛ
    # Если пересчитываем шаг по времени нужно перезапустить этот шаг сначала (делаем Indicator = False)
    # Если условие КФЛ продолжает выполняться то делаем Indicator = True
    # Если индикатор со значением True выживет до конца шага цикла, то мы должны перейти на следующий временной слой
    if i==1:
        dt_new = (C * ((r_phys[1:]-r_phys[:-1]) * pk_to_meter) / (a + (np.abs(U[1,1:])))).min()
    else:
        dt_new = 0.1*((C * ((r_phys[1:]-r_phys[:-1]) * pk_to_meter) / (a + (np.abs(U[1,1:])))).min())

    if (dt_new < dt):
        print(dt, '$$', i)
        dt = dt_new
        indicator = False
        indicator_2 = True
        print(dt, '&&', i)
        # print('U_max',U.max())
        continue

    elif (dt_new >= dt):
        indicator = True
        # Следующий условный оператор нужен чтобы увеличить шаг по времени после пересчета первого шага
        # Это нужно так как при первом пересчете шаг по времени получается излишне маленьким
        if indicator_2 == True and i == 1:
            dt = dt_new * dt_minimizer
            print('aaa', dt)
            indicator_2 = False
            continue


    # Обновление давления
    P[1,:] =  (gamma - 1) * (Rho[1,:]) * M_unit * 3/2 * k/M_unit * T / m_p

    # Пересчет гравитационного потенциала (аккреция)
    # M_pbh_accreted = (V[1] * Rho_mean * np.abs((dt * Rho[0,1] * U_half[1] * S[1]) / V[1]) * (U_half[1]<0))/M_sun
    # M_pbh += M_pbh_accreted
    # Phi[:-1] -= -G * M_pbh_accreted * M_sun/(r_temp * pk_to_meter)

    # Если индикатор равен True, то идем на следующий временной слой, если равен False, то повторяем текущий
    # Записываем в массив все значения шага по времени которые не вызвали пересчет слоя
    if (indicator == True):
        # Увеличиваем время симуляции
        sim_time += dt

        # Сохраняем значения массивов через draw_step для последующей отрисовки
        if (i%draw_step == 0):
            U_draw = np.vstack([U_draw, U[1,:]])
            Rho_draw = np.vstack([Rho_draw, Rho[1,:]])
            E_draw = np.vstack([E_draw, E[1,:]])
            draw_time = np.append(draw_time, sim_time)
            # Далее под катом идет 3 отладочных блока для вывода различных параметров
            # print('Rho_draw[0] =', Rho_draw[Rho_draw.shape[0]-1,0])
            # print('Rho_draw[1] =', Rho_draw[Rho_draw.shape[0]-1,1])
            # print('Rho_draw[2] =', Rho_draw[Rho_draw.shape[0]-1,2])
            # print('Rho_draw[3] =', Rho_draw[Rho_draw.shape[0]-1,3])
            # print('Rho_draw[4] =', Rho_draw[Rho_draw.shape[0]-1,4])

            # print('U_half[0] =', U_half[0])
            # print('U_half[1] =', U_half[1])
            # print('U_half[2] =', U_half[2])
            # print('U_half[3] =', U_half[3])
            # print('U_half[4] =', U_half[4])

            # print('U_draw[0] =', U_draw[U_draw.shape[0]-1,0])
            # print('U_draw[1] =', U_draw[U_draw.shape[0]-1,1])
            # print('U_draw[2] =', U_draw[U_draw.shape[0]-1,2])
            # print('U_draw[3] =', U_draw[U_draw.shape[0]-1,3])
            # print('U_draw[4] =', U_draw[U_draw.shape[0]-1,4])

            print('Номер шага ', i)
            print('Прошло ', sim_time * 3.169e-8, 'лет')

        # Присвоение новых значений массивам
        U[0,:] = U[1,:]
        Rho[0,:] = Rho[1,:]
        E[0,:] = E[1,:]
        P[0,:] = P[1,:]

        Rho_dump_left[0] = Rho_dump_left[1]

        U[1,:] = 0
        Rho[1, :] = 0
        E[1, :] = 0
        P[1, :] = 0
        i = i + 1

print('Общее время симуляции равно', sim_time * 3.169e-8, 'лет')

262652610925.36276 $$ 1
1788138459.813231 && 1
aaa 236376468.64523482
Номер шага  10000
Прошло  74907.70291367755 лет
Номер шага  20000
Прошло  149815.4058273058 лет


KeyboardInterrupt: 

In [ ]:
# n=80 t=11696 Rho=1719
# n=70 t=11326 Rho=1632
# n=60 t=11656 Rho=1476`
# n=50 t=11117 Rho=1382
# n=40 t=11569 Rho=1202
# n=30 t=10591 Rho=1111
# n=20 t=11252 Rho=912
#
# 100 10255 2091 : 200 10085 2995 :
# 20 11252 912 : 21 10622 963 : 22 10059 1015 : 23 9552 1067 : 24 -- : 25 10847 1013 : 26 10372 1060

In [ ]:
Rho_draw

In [ ]:
# Отрисовка плотности
import matplotlib.pyplot as plt
Font = 16

#Задаем начальное значение и шаг по времени для отрисовки графиков
draw_time_zero = 1
draw_time_step = 1

fig, ax = plt.subplots()

for t in range(draw_time_zero, Rho_draw.shape[0], draw_time_step):
    ax.plot(r_phys[:]*1e-3, Rho_draw[t, :], label=f"t = {int(draw_time[t] * 3.169e-8 * 1e-3)} тыс.лет")
ax.legend(fontsize = Font)
ax.set_xlabel(r"r, кпк",fontsize = Font)
ax.set_ylabel(r"$\rho,$ $\bar\rho$",fontsize = Font)

ax.set_xscale('log')
# ax.set_yscale('log')
# ax.set_ylim(-1, 0.5)
# ax.set_xlim(0,9)

#Добавляем вертикальные линии в точках r_virial и r_s
plt.vlines(x=[r_virial*1e-3, r_s*1e-3], ymin=0.1, ymax=Rho_draw.max(), colors=['red', 'blue'], linestyles='dashed')

plt.text(x=r_virial*1e-3+0.001, y=0.1, s=r"$r_{virial}$", color='red', rotation=90)
plt.text(x=r_s*1e-3+0.001, y=0.1, s=r"$r_{stop}$", color='blue', rotation=90)

plt.show()
plt.savefig('Rho_1.eps')

In [ ]:
# Отрисовка плотности

# импортируем библиотеку matplotlib
import matplotlib.pyplot as plt

#Задаем начальное значение и шаг по времени для отрисовки графиков
draw_time_zero = 0
draw_time_step = 1

# создаем фигуру и оси
fig, ax = plt.subplots()

# перебираем значения draw_time от 0 до length_t-1 с шагом draw_time_step
for t in range(draw_time_zero, U_draw.shape[0], draw_time_step):
    # рисуем график значений Rho[t, :] и r[:] с подписью t
    ax.plot(r_phys, U_draw[t, :], label=f"t = {draw_time[t] * 3.169e-8}")

# добавляем легенду
ax.legend()
# ax.set_ylim(2500, 3500)
# ax.set_xlim(10,12)

# показываем фигуру
plt.show()